In [11]:
import read_files_v2, manage_data
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [12]:
path_to_folder = "/eos/user/j/jcapotor/FBGdata/Data/PatchCableTests/20230209/"
files = ["1", "2", "3", "4", "5", "6", "7", "8"]

In [13]:
def line(x, A, B):
    return A+B*x

In [ ]:
amp_results = {}
wav_results = {}
fwhm_results = {}
residuals_results = {}
baseline_results = {}
for file in files:
    try:
        print("Filling Run" + file)
        peaks = read_files.read_peaks(path_to_folder + "Run"+file+"_peaks")
        peaks_p, peaks_s, peaks_p_full, peaks_s_full = manage_data.process_data(peaks)
        spectrums = read_files.read_spectrums(path_to_folder + "Run"+file+"_specs", [0])
        spectrums_p, spectrums_s, spectrums_p_full, spectrums_s_full = manage_data.process_data(spectrums)
        spectrums_p, peaks_p = manage_data.match_dataframes(spectrums_p, peaks_p)

        amp_results["Run"+file] = {}
        wav_results["Run"+file] = {}
        fwhm_results["Run"+file] = {}
        residuals_results["Run"+file] = {}
        baseline_results["Run"+file] = np.mean(spectrums_p["Baseline"])
        for variable in ["Wav1-1", "Wav1-2", "Wav1-3", "Wav1-4"]:
            amp_results["Run"+file]["specs_P_"+variable+"_amp (AU)"] = str(np.round(np.mean(spectrums_p[variable+"_amp"]), 0)) + " +- " + str(np.round(np.std(spectrums_p[variable+"_amp"]),0))
            wav_results["Run"+file]["peaks_P_"+variable] = str(np.round(np.mean(peaks_p[variable]*1e9), 6)) + " +- " + str(np.round(np.std(peaks_p[variable]*1e9), 6)) + " nm"
            wav_results["Run"+file]["specs_P_"+variable] = str(np.round(np.mean(spectrums_p[variable]), 6)) + " +- " + str(np.round(np.std(spectrums_p[variable]), 6)) + " nm"
            fwhm_results["Run"+file]["specs_P_"+variable+"_sigma"] = str(np.round(np.mean(spectrums_p[variable+"_sigma"]), 6)) + " +- " + str(np.round(np.std(spectrums_p[variable+"_sigma"]), 6)) + " nm"
            popt_specs, pcov_specs = curve_fit(line, spectrums_p["Timestamp"], spectrums_p[variable])
            popt_peaks, pcov_peaks = curve_fit(line, peaks_p["Timestamp"], peaks_p[variable]*1e9)
            res_specs = line(spectrums_p["Timestamp"], popt_specs[0], popt_specs[1]) - spectrums_p[variable]
            res_peaks = line(peaks_p["Timestamp"], popt_peaks[0], popt_peaks[1]) - peaks_p[variable]*1e9
            
            residuals_results["Run"+file]["peaks_P"+variable+"_residuals"] = (np.round(np.std(res_peaks*1e6), 1))
            residuals_results["Run"+file]["spectrums_P"+variable+"_residuals"] = (np.round(np.std(res_specs*1e6), 1))
    except:
        continue

In [15]:
print("Residuals (fm)")
residuals = pd.DataFrame(residuals_results).T
print(residuals)

Residuals (fm)
      peaks_PWav1-1_residuals  spectrums_PWav1-1_residuals  \
Run1                    112.8                        131.8   
Run2                     15.3                         24.5   
Run3              384634203.6                         41.2   
Run4                      NaN                          NaN   
Run5                      0.0                         56.8   
Run6                      NaN                          NaN   
Run7                      0.0                       4482.2   

      peaks_PWav1-2_residuals  spectrums_PWav1-2_residuals  \
Run1                    176.5                        191.7   
Run2                     17.6                         28.4   
Run3                1271812.8                         45.0   
Run4                      NaN                          NaN   
Run5                      0.0                        430.8   
Run6                      NaN                          NaN   
Run7                      0.0                         

In [18]:
residuals = pd.DataFrame(wav_results).T
print(residuals)
print("Comparison of I4G peak - Spectrums peak (fm)")
comparison = {}
for index, row in residuals.iterrows():
    comparison[index] = {}
    for i in range(1,5):
        try:
            peaks_name = "peaks_P_Wav1-"+str(i)
            specs_name = "specs_P_Wav1-"+str(i)
            peaks_value = float(row[peaks_name].split(" ")[0])
            specs_value = float(row[specs_name].split(" ")[0])
            comparison[index]["Wav1-"+str(i)+" (fm)"] = (specs_value - peaks_value)*1e6
        except:
            continue
print(pd.DataFrame(comparison).T)

                  peaks_P_Wav1-1              specs_P_Wav1-1  \
Run1   1540.031042 +- 0.00014 nm  1540.031365 +- 0.000159 nm   
Run2   1540.028434 +- 7.4e-05 nm   1540.028736 +- 5.4e-05 nm   
Run3  339.730011 +- 595.35653 nm   1540.02738 +- 0.000101 nm   
Run4               0.0 +- 0.0 nm  1540.037889 +- 0.000322 nm   
Run5               0.0 +- 0.0 nm  1540.032982 +- 0.000201 nm   
Run6               0.0 +- 0.0 nm   1539.705255 +- 0.20039 nm   
Run7               0.0 +- 0.0 nm   1540.02705 +- 0.004875 nm   

                  peaks_P_Wav1-2              specs_P_Wav1-2  \
Run1  1545.122736 +- 0.000296 nm   1545.12336 +- 0.000335 nm   
Run2   1545.121536 +- 3.7e-05 nm   1545.122366 +- 5.7e-05 nm   
Run3  1541.151186 +- 1.969945 nm  1545.123342 +- 0.000198 nm   
Run4                         NaN                         NaN   
Run5               0.0 +- 0.0 nm   1545.133542 +- 0.00101 nm   
Run6                         NaN                         NaN   
Run7               0.0 +- 0.0 nm   1545

In [19]:
print(pd.DataFrame(amp_results).T)

     specs_P_Wav1-1_amp (AU) specs_P_Wav1-2_amp (AU) specs_P_Wav1-3_amp (AU)  \
Run1          4146.0 +- 23.0          3965.0 +- 44.0          3618.0 +- 40.0   
Run2          1183.0 +- 30.0           889.0 +- 41.0           692.0 +- 86.0   
Run3          3915.0 +- 10.0          3912.0 +- 19.0          3651.0 +- 29.0   
Run4            223.0 +- 4.0                     NaN                     NaN   
Run5            279.0 +- 2.0            231.0 +- 0.0            211.0 +- 1.0   
Run6            354.0 +- 1.0                     NaN                     NaN   
Run7            210.0 +- 0.0            197.0 +- 0.0            176.0 +- 0.0   

     specs_P_Wav1-4_amp (AU)  
Run1          2905.0 +- 35.0  
Run2          706.0 +- 107.0  
Run3          2786.0 +- 13.0  
Run4                     NaN  
Run5           188.0 +- 11.0  
Run6                     NaN  
Run7            188.0 +- 0.0  


In [20]:
print("Sigma of gaussian fit (pm)")
print(pd.DataFrame(fwhm_results).T)

Sigma of gaussian fit (pm)
         specs_P_Wav1-1_sigma     specs_P_Wav1-2_sigma  \
Run1  0.063906 +- 0.006202 nm  0.062314 +- 0.005304 nm   
Run2  0.107367 +- 0.011878 nm  0.069013 +- 0.005701 nm   
Run3  0.085108 +- 0.007169 nm  0.082284 +- 0.005962 nm   
Run4  0.001072 +- 0.007197 nm                      NaN   
Run5   0.01118 +- 0.001072 nm  0.008984 +- 0.016326 nm   
Run6  0.006476 +- 0.002917 nm                      NaN   
Run7   0.010859 +- 0.00255 nm  0.012721 +- 0.000107 nm   

         specs_P_Wav1-3_sigma      specs_P_Wav1-4_sigma  
Run1  0.075623 +- 0.011656 nm   0.050721 +- 0.003618 nm  
Run2  0.052531 +- 0.003829 nm   0.063677 +- 0.004355 nm  
Run3  0.064113 +- 0.003031 nm   0.057135 +- 0.002471 nm  
Run4                      NaN                       NaN  
Run5  0.005723 +- 0.012725 nm  -0.003106 +- 0.018819 nm  
Run6                      NaN                       NaN  
Run7     0.013338 +- 4e-05 nm      0.0111 +- 4.9e-05 nm  


In [23]:
print(spectrums["Data"])

0     [136, 119, 101, 90, 80, 80, 79, 88, 97, 115, 1...
1     [195, 218, 241, 261, 279, 292, 304, 309, 312, ...
2     [134, 116, 100, 88, 80, 77, 79, 86, 98, 115, 1...
3     [200, 224, 245, 264, 280, 294, 303, 310, 310, ...
4     [135, 115, 101, 87, 81, 76, 80, 84, 100, 115, ...
5     [198, 222, 244, 266, 281, 296, 304, 311, 311, ...
6     [134, 115, 100, 87, 80, 77, 80, 86, 100, 116, ...
7     [201, 225, 247, 266, 283, 296, 304, 310, 310, ...
8     [131, 113, 97, 86, 78, 78, 80, 90, 102, 120, 1...
9     [202, 225, 248, 266, 285, 296, 307, 310, 312, ...
10    [133, 116, 98, 88, 80, 78, 79, 89, 100, 120, 1...
11    [201, 224, 247, 266, 284, 296, 306, 311, 312, ...
12    [130, 111, 96, 84, 78, 77, 82, 90, 104, 122, 1...
13    [204, 228, 249, 270, 285, 299, 306, 312, 310, ...
14    [131, 111, 98, 84, 80, 76, 82, 88, 104, 120, 1...
15    [206, 230, 252, 272, 287, 300, 308, 312, 311, ...
16    [128, 110, 95, 84, 78, 76, 80, 90, 104, 123, 1...
17    [206, 229, 251, 270, 287, 299, 308, 312, 3